<a href="https://colab.research.google.com/github/benchov/AlgoTrading/blob/master/ALPACA_Statistical_Arbitrage_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
!pip install alpaca-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
# imports
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass, AssetStatus
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime

In [93]:
# API keys for Alpaca
API_KEY = 'PKJAMRGMC0EMO7LCX999'
SECRET_KEY = 'gRhoCnuCZ7qpzQkH1NRSMVQZXNGyNcurvrexeJEp'

In [94]:
# creating an instance of client
trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)

In [95]:
# test the availability of the api by account information
account = trading_client.get_account()
print(account)

id=UUID('d38ec6c2-a23b-4708-a064-cda26744e218') account_number='PA3OJP423MLE' status=<AccountStatus.ACTIVE: 'ACTIVE'> crypto_status=<AccountStatus.ACTIVE: 'ACTIVE'> currency='USD' buying_power='200000' regt_buying_power='200000' daytrading_buying_power='0' non_marginable_buying_power='100000' cash='100000' accrued_fees='0' pending_transfer_out=None pending_transfer_in='0' portfolio_value='100000' pattern_day_trader=False trading_blocked=False transfers_blocked=False account_blocked=False created_at=datetime.datetime(2022, 12, 22, 19, 56, 32, 73526, tzinfo=datetime.timezone.utc) trade_suspended_by_user=False multiplier='2' shorting_enabled=True equity='100000' last_equity='100000' long_market_value='0' short_market_value='0' initial_margin='0' maintenance_margin='0' last_maintenance_margin='0' sma='100000' daytrade_count=0


In [96]:
# get the available US equity stocks
search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY, status=AssetStatus.ACTIVE)
assets = trading_client.get_all_assets(search_params)
asset_list = []
for i in assets:
  if i.shortable == True and i.tradable == True and i.fractionable == True:
    asset_list.append(i.symbol)
asset_list[:5],len(asset_list)

(['NTAP', 'NTB', 'NTCT', 'PCH', 'PCTY'], 3849)

In [97]:
assets[0]

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'easy_to_borrow': False,
    'exchange': <AssetExchange.OTC: 'OTC'>,
    'fractionable': False,
    'id': UUID('9f3d7508-c70e-486c-92f8-4079224b609c'),
    'maintenance_margin_requirement': 100.0,
    'marginable': False,
    'min_order_size': None,
    'min_trade_increment': None,
    'name': 'ALFI INC Common Shares',
    'price_increment': None,
    'shortable': False,
    'status': <AssetStatus.ACTIVE: 'active'>,
    'symbol': 'ALFIQ',
    'tradable': False}

#### Get asset price history - 200 steps back into past.


1.   Createing a function which get the filtered available assets
2.   Get only those which has enough data (for example enough day to the strategy can calculate the cointegration
3.   Save into a DataFrame





In [90]:
# Get data from ALPACA server
symbol_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)
multi_symbol_params = StockBarsRequest(symbol_or_symbols=asset_list[:1000], 
                                        timeframe=TimeFrame.Hour,
                                        start="2023-01-01 00:00:00"
                                        )
 
bars = symbol_client.get_stock_bars(multi_symbol_params)

# turn data into DataFrame
df_bars = bars.df

In [101]:
tickers = df_bars.index.get_level_values(0).unique()

(983, 983)

In [99]:
# removing those tickers which has not enough data to calculate cointegration
symbol_for_remove = []
for symbol in tickers:
  if len(df_bars.loc[symbol]) < 200:
      symbol_for_remove.append(symbol)
df_bars_cleared = df_bars.drop(symbol_for_remove, axis=0)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [105]:
print(f"{len(tickers) - len(symbol_for_remove)} ticker removed")

297 ticker removed


#### Calculating cointegration